In [ ]:
!pip install wandb

Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tqdm import tqdm
import numpy as np
import wandb
from wandb.keras import WandbCallback

In [ ]:
configs = {
            "learning_rate": 0.001,
            "epochs": 24,
            "batch_size": 32,
            "log_step": 200,
            "val_log_step": 50
           }
run = wandb.init(project='Face Recognition', config=configs)
config = wandb.config

Hyper Parameters

In [ ]:
batch_size = 32
epochs = 24
width = height = 224

Load and Split Data

In [ ]:
PATH = '/content/drive/MyDrive/Dataset/7-7 dataset'

idg = ImageDataGenerator(
    rescale = 1.0 / 255.0,
    horizontal_flip=True,
    brightness_range=(0.7, 1.2),
    zoom_range=0.2,
    validation_split=0.2
)

train_data = idg.flow_from_directory(
    PATH,
    class_mode='categorical',
    target_size=(width, height),
    batch_size=batch_size,
    subset='training'
)

val_data = idg.flow_from_directory(
    PATH,
    class_mode='categorical',
    target_size=(width, height),
    batch_size=batch_size,
    subset='validation'
)

AliNet Class

In [ ]:
class AliNet(Model):
    def __init__(self):
        super().__init__()
        
        self.conv2d_1 = Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, 3))
        self.conv2d_2 = Conv2D(64, (3, 3), activation='relu')
        self.conv2d_3 = Conv2D(128, (3, 3), activation='relu')
        self.maxpool2d = MaxPooling2D()
        self.flatten = Flatten()
        self.dense_1 = Dense(256, activation='relu')
        self.dense_2 = Dense(128, activation='relu')
        self.dense_3 = Dense(14, activation='softmax')
        self.dropout = Dropout(0.3)

    def call(self, x):
        y = self.conv2d_1(x)
        y = self.maxpool2d(y)
        # y = self.conv2d_2(y)
        # y = self.maxpool2d(y)
        y = self.conv2d_3(y)
        y = self.maxpool2d(y)
        y = self.flatten(y)
        # y = self.dense_1(y)
        # y = self.dropout(y)
        y = self.dense_2(y)
        y = self.dropout(y)
        out = self.dense_3(y)

        return out

In [8]:
model = AliNet()

In [9]:
optimizer = tf.keras.optimizers.Adam(config.learning_rate)
loss_function = tf.keras.losses.CategoricalCrossentropy()

In [10]:
train_loss = tf.keras.metrics.MeanAbsoluteError()
val_loss = tf.keras.metrics.MeanAbsoluteError()
train_accuracy = tf.keras.metrics.CategoricalAccuracy()
val_accuracy = tf.keras.metrics.CategoricalAccuracy()

Train Loop

In [11]:
for epoch in range(epochs):
    train_accuracy.reset_states()
    val_accuracy.reset_states()
    train_loss.reset_states()
    val_loss.reset_states()
    print("Epochs:", epoch)

    # Training ----------------------------------------------------------
    for idx, (images, labels) in enumerate(tqdm(train_data)):
        with tf.GradientTape() as gTape:
            predictions = model(images)
            loss = loss_function(labels, predictions)

            train_loss(labels, predictions)
            train_accuracy(labels, predictions)

        # محاسبات مشتق ها
        gradients = gTape.gradient(loss, model.trainable_variables)

        # به روز رسانی وزن ها
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        if len(train_data) <= idx:
            break

    # Validation --------------------------------------------------------
    for idx, (images, labels) in enumerate(tqdm(val_data)):
        predictions = model(images)
        loss = loss_function(labels, predictions)

        val_loss(labels, predictions)
        val_accuracy(labels, predictions)

        if len(val_data) <= idx:
            break

    print('Train Accuracy:', train_accuracy.result())
    print('Validation Accuracy:', val_accuracy.result())
    print('Train Loss:', train_loss.result())
    print('Validation Loss:', val_loss.result())

    wandb.log({
        'epochs': epoch,
        'acc': float(train_accuracy.result()),
        'loss': np.mean(train_loss.result()),
        'val_acc':float(val_accuracy.result()),         
        'val_loss': np.mean(val_loss.result())
        })

Epochs: 0


100%|██████████| 9/9 [03:06<00:00, 20.77s/it]


Train Accuracy: tf.Tensor(0.19607843, shape=(), dtype=float32)
Validation Accuracy: tf.Tensor(0.40333334, shape=(), dtype=float32)
Train Loss: tf.Tensor(0.12685081, shape=(), dtype=float32)
Validation Loss: tf.Tensor(0.10875356, shape=(), dtype=float32)
Epochs: 1


100%|██████████| 9/9 [00:09<00:00,  1.10s/it]


Train Accuracy: tf.Tensor(0.5900178, shape=(), dtype=float32)
Validation Accuracy: tf.Tensor(0.57666665, shape=(), dtype=float32)
Train Loss: tf.Tensor(0.08279116, shape=(), dtype=float32)
Validation Loss: tf.Tensor(0.07359373, shape=(), dtype=float32)
Epochs: 2


...

Train Accuracy: tf.Tensor(0.9768271, shape=(), dtype=float32)
Validation Accuracy: tf.Tensor(0.80333334, shape=(), dtype=float32)
Train Loss: tf.Tensor(0.00613341, shape=(), dtype=float32)
Validation Loss: tf.Tensor(0.029845448, shape=(), dtype=float32)
Epochs: 22


100%|██████████| 9/9 [00:09<00:00,  1.09s/it]


Train Accuracy: tf.Tensor(0.9768271, shape=(), dtype=float32)
Validation Accuracy: tf.Tensor(0.89, shape=(), dtype=float32)
Train Loss: tf.Tensor(0.005021341, shape=(), dtype=float32)
Validation Loss: tf.Tensor(0.020482942, shape=(), dtype=float32)
Epochs: 23


100%|██████████| 9/9 [00:09<00:00,  1.09s/it]

Train Accuracy: tf.Tensor(0.9821747, shape=(), dtype=float32)
Validation Accuracy: tf.Tensor(0.84, shape=(), dtype=float32)
Train Loss: tf.Tensor(0.0046032723, shape=(), dtype=float32)
Validation Loss: tf.Tensor(0.024605714, shape=(), dtype=float32)


Save Model

In [ ]:
model.save('AliFaceRecognition.h5')
model.save_weights(filepath='faceRecognition', save_format='HDF5')